# Description

This notebook will make sure all the conditions are met to rebin the files

 * check that the **binning_list_of_files** file has been created
 * check that the folder structure of the image corrected is correct
  * right structure
  * number of file in each folder is identical
 * check that all the files reported in the binning_list_of_files exist
  * if they do not, user will have the option to fix the issue himself, or to automatically fix the binning_list_of_files file
  
Please select **Cell > Run All** to run this notebook

# Import

In [5]:
import os
import pandas as pd
from IPython.core.display import display, HTML
from ipywidgets import widgets

import config_handler
import utilities

# Checking  

Global dictionary that will keep record of the various checks

In [6]:
global_status = {}
global_status['binning_list_of_files'] = False
global_status['all_images_folder_found'] = False
global_status['all_folder_have_same_number_of_files'] = False

## binning_list_of_files exists ?

In [7]:
root_folder = config_handler.load_config(key = 'output_folder')
binning_list_of_files = os.path.join(root_folder, 'binning_list_of_files.csv')
if os.path.exists(binning_list_of_files):
    global_status['binning_list_of_files'] = True

## folder structure is correct ?

### each corrected image requested by rebin has its own folder ?

In [9]:
if global_status['binning_list_of_files']:
    input_raw_folder = config_handler.load_config(key = 'input_corrected_folder')
    
    df = pd.read_csv(binning_list_of_files, index_col=0, header=None)
    
    list_images = []
    for _files in df.loc['list_of_files',:]:
        list_images.append(_files.split(','))
        
    folder_not_found = []
    for _images_in_bin in list_images:
        for _image in _images_in_bin:
            _image_number = utilities.keep_folder_name(_image)
            _full_image_path = os.path.join(input_raw_folder, _image_number)
            if not os.path.exists(_full_image_path):
                folder_not_found.append(_image_number)
    
    if folder_not_found == []:
        global_status['all_images_folder_found']=True

### all folder have the same number of files ?

In [10]:
if global_status['binning_list_of_files']:
    
    number_of_files = []
    for _images_in_bin in list_images:
        for _image in _images_in_bin:
            _image_number = utilities.keep_folder_name(_image)
            _full_image_path = os.path.join(input_raw_folder, _image_number)
            if os.path.exists(_full_image_path):
                _nbr_files = len([name for name in os.listdir(_full_image_path) \
                                  if (os.path.isfile(os.path.join(_full_image_path, name)) and \
                                                     (utilities.is_extension(name, '.fits')))])
                number_of_files.append(_nbr_files)
    number_of_files = set(number_of_files)
    if len(number_of_files) == 1:
        global_status['all_folder_have_same_number_of_files'] = True

# Results

In [11]:
if global_status['binning_list_of_files']:
    display(HTML('<span style="font-size: 20px">- binning_list_of_files found? ........  <span style="color:green">OK!</span></span>'))
    if global_status['all_images_folder_found']:
        display(HTML('<span style="font-size: 20px">- all images requested have their corrected folder? ........<span style="color:green">OK!</span></span>'))
    else:
        display(HTML('<span style="font-size: 20px">- all images requested have their corrected folder? ........<span style="color:red">FAILED!</span></span>'))
        display(HTML('<pre><span style="font-size: 18px">    List of Folder(s) Missing:</span></pre>'))
        for _folder in folder_not_found:
            display(HTML('<pre><span style="fonts-size: 18px">         ' + _folder + '</span></pre>'))
        display(HTML('<pre><span style="font-size: 18px">    <span style="font-weight: bold;">How to Fix Issue:</span>'))
        print("\t 1. Fix issue yourself!")
        print("\t or")
        print("\t 2. Select Fix Image Folder Not Found Issue from bottom drop-downlist and run the CHECK ISSUE cell!")

    if global_status['all_folder_have_same_number_of_files']:
        display(HTML('<span style="font-size: 20px">- all folders of images corrected have the same number of files? ........<span style="color:green">OK!</span></span>'))
    else:
        display(HTML('<span style="font-size: 20px">- all folders of images corrected have the same number of files? ........<span style="color:red">FAILED!</span></span>'))
        display(HTML('<pre><span style="font-size: 18px">    <span style="font-weight: bold;">How to Fix Issue:</span>'))
        print("\t 1. Fix issue yourself!")
        print("\t or")
        print("\t 2. Select Get Complete Report on Number of Images per Folder from bottom drop-downlist and run CHECK ISSUE")
else:
    display(HTML('<span style="font-size: 20px">- binning_list_of_files found? ........  \
                 <span style="color:red">FAILED!</span></span>'))
    display(HTML('<span style="font-size: 20px">------> <span style="font-weight: bold;">How to Fix Issue:</span> run step4_making_time_binning_list_of_files</span>'))

# Solutions 

In [7]:
def run_selection():
    print("here")

In [8]:
from IPython.display import display
fix_algo = ['None', 'Fix Image Folder Not Found Issue!', 'Get Complete Report on Number of Images per Folder']
fix = widgets.Dropdown(options=fix_algo,
                       description='Select your Solutions')
display(fix)

In [9]:
if fix.value == 'Fix Image Folder Not Found Issue!':
    print("NOT IMPLEMENTED YET!")
elif fix.value == 'Get Complete Report on Number of Images per Folder':
    dictionary_list_of_files = {}
    dictionary_list_of_files['folder_name'] = []
    dictionary_list_of_files['number_of_files'] = []
    
    folder_name_array = []
    number_of_files_array = []
    
    for _images_in_bin in list_images:
        for _image in _images_in_bin:
            _image_number = utilities.keep_folder_name(_image)
            _full_image_path = os.path.join(input_raw_folder, _image_number)
            if os.path.exists(_full_image_path):
                _nbr_files = len([name for name in os.listdir(_full_image_path) \
                                  if (os.path.isfile(os.path.join(_full_image_path, name)) and \
                                                     (utilities.is_extension(name, '.fits')))])
                folder_name_array.append(_image_number)
                number_of_files_array.append(_nbr_files)

    dictionary_list_of_files['folder_name'] = folder_name_array
    dictionary_list_of_files['number_of_files'] = number_of_files_array
    
for _index, _folder_name in enumerate(folder_name_array):
    print("{}: {}".format(_folder_name, dictionary_list_of_files['number_of_files'][_index]))

NameError: name 'folder_name_array' is not defined